In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn import *
import nltk, datetime
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
train = pd.read_csv('../input/sales_train.csv')
test = pd.read_csv('../input/test.csv')
submission = pd.read_csv('../input/sample_submission.csv')
items = pd.read_csv('../input/items.csv')
item_cats = pd.read_csv('../input/item_categories.csv')
shops = pd.read_csv('../input/shops.csv')

print(train.shape , test.shape)
print(train[:3])
print(test[:3])

In [ ]:
#difference between train and test
#[c for c in train.columns if c not in test.columns]
#another good way above mentione to run loop in python
a = []
for c in train.columns:
    if c not in test.columns:
        #print(c)
        a = c
print(a)


In [ ]:
train.head() # train is loaded as dataframe using pandas read.csv

In [ ]:
test.head()

In [ ]:
items.head()

In [ ]:
#bag of words
feature_count = 25
#creation of bag of words using tokenization technique(i.e extraction of words seperated by delimeter here 'space')
#one hot encoding
text_feature_frame= feature_extraction.text.TfidfVectorizer(max_features=feature_count)  #metadata about the vectoried text
items['items_name_len'] = items['item_name'].map(len) 
print(items['items_name_len'])
items['items_name_wc'] = items['item_name'].map(lambda x : len(str(x).split(' ')))
print(items['items_name_wc'])

In [ ]:
print(text_feature_frame) #metadata about the vectoried text

In [ ]:
#counting
text_feature = text_feature_frame.fit_transform(items['item_name']).toarray()  #one hot encoding
#print(text_feature)
text_feature = pd.DataFrame(text_feature)
print(text_feature)

In [ ]:
col = text_feature.columns
for i in range(feature_count):
    items['item_name_trnsfrmindf_' +str(i)] = text_feature[col[i]]

In [ ]:
items.head()

In [ ]:
# for item_cats
feature_count = 25
text_feature_frame= feature_extraction.text.TfidfVectorizer(max_features=feature_count)  #metadata about the vectoried text  #object form 
item_cats['item_category_name_len'] = item_cats['item_category_name'].map(len) 
print(item_cats['item_category_name_len'])
item_cats['item_category_name_wc'] =item_cats['item_category_name'].map(lambda x : len(str(x).split(' ')))
print(item_cats['item_category_name_wc'])

In [ ]:
text_Features = pd.DataFrame(text_feature_frame.fit_transform(item_cats['item_category_name']).toarray()) #text_feature_frame is an object , #text_Features is a data frame holding one hot encoding
#print(text_Features)
col = text_Features.columns #again col is an object holding 25 col data
for i in range(feature_count):
    item_cats['item_category_name_tfidf_' + str(i)] = text_Features[col[i]]

In [ ]:
item_cats.head()

In [ ]:
#shops
feature_count = 25
text_feature_frame= feature_extraction.text.TfidfVectorizer(max_features=feature_count)  #metadata about the vectoried text  #object form 
shops['shop_name_len'] = shops['shop_name'].map(len) 
print(shops['shop_name_len'])
shops['shop_name_wc'] =shops['shop_name'].map(lambda x : len(str(x).split(' ')))
print(shops['shop_name_wc'])


In [ ]:
text_Features = pd.DataFrame(text_feature_frame.fit_transform(shops['shop_name']).toarray()) #text_feature_frame is an object , #text_Features is a data frame holding one hot encoding
#print(text_Features)
col = text_Features.columns #again col is an object holding 25 col data
for i in range(feature_count):
    shops['shop_name_tfidf_' + str(i)] = text_Features[col[i]]

In [ ]:
shops.head()

In [ ]:
#train
train.head()

In [ ]:
train['date'] = pd.to_datetime(train['date'], format='%d.%m.%Y')

In [ ]:
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
#drop by default works on rows , for columns axis =1 
train = train.drop(['date','item_price'], axis=1)

In [ ]:
train.head()

In [ ]:
train = train.groupby([c for c in train.columns if c not in 'item_cnt_day'], as_index=False)['item_cnt_day'].sum()  #as_index=false means grouped by elements are no longer index in dataframe
#[c for c in train.columns if c not in ['item_cnt_day']]
train.head()
#now rename item_cnt_day to item_cnt_month
train = train.rename(columns={'item_cnt_day':'item_cnt_month'})


In [ ]:
#findng monthly mean
monthly_mean_shop_item = train[['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'], as_index=False)['item_cnt_month'].mean()
#print(monthly_mean_shop_item)
monthly_mean_shop_item = monthly_mean_shop_item.rename(columns={'item_cnt_month':'month_mean_shop_item'})
print(monthly_mean_shop_item)
#finding mean
train = pd.merge(train,monthly_mean_shop_item ,how = "left" , on = ['shop_id','item_id'])


In [ ]:
train.tail(10)

In [ ]:
#Last Month 
#let`s work with year 2015(10) datablock no 33
shop_item_prev_month = train[train['date_block_num'] == 33][['shop_id','item_id','item_cnt_month']]
shop_item_prev_month = shop_item_prev_month.rename(columns={'item_cnt_month':'item_cnt_prev_month'})
print(shop_item_prev_month)


In [ ]:
#Add previous month features with train data
#also do some cleaning by replacing all nan value or null values
train = pd.merge(train,shop_item_prev_month, how='left',on=['shop_id','item_id']).fillna(0.)


In [ ]:
train.tail()

In [ ]:
items.head()

In [ ]:
train = pd.merge(train,items,how='left',on=['item_id'])

In [ ]:
train.tail()

In [ ]:
item_cats.tail()

In [ ]:
#listing all column headers of train df
list(train.columns.values)

In [ ]:
train = pd.merge(train , item_cats , how='left',on=[ 'item_category_id'])

In [ ]:
train.tail()

In [ ]:
#shop features merge to train df
train = pd.merge(train , shops, how = 'left' , on=['shop_id'])

In [ ]:
train.tail()

In [ ]:
#playing with test dataset
test.tail()


In [ ]:
monthly_mean_shop_item .tail()

In [ ]:
shop_item_prev_month.tail()

In [ ]:
items.tail()

In [ ]:
test['month']=11
test['year'] = 2015
test['date_block_num'] =34
test.tail()


In [ ]:
#till oct 2015 we have data in train data set now we need to predict for nov 2015 sales for each product
test = pd.merge(test,monthly_mean_shop_item ,how='left',on=['shop_id','item_id']).fillna(0.)
test = pd.merge(test,shop_item_prev_month,how='left',on=['shop_id','item_id']).fillna(0.)
test = pd.merge(test,items,how='left',on=['item_id'])
test = pd.merge(test,item_cats,how='left',on=['item_category_id'])
test = pd.merge(test,shops,how='left',on=['shop_id'])
#test['item_cnt_month']=0.



In [ ]:
test.tail()

In [ ]:
test['item_cnt_month']=0.

In [ ]:
test['item_cnt_month'].values


In [ ]:
test.tail()

In [ ]:
list(test.columns.values)

In [ ]:
list(train.columns.values)

In [ ]:
#visualization
import matplotlib.pyplot as plt #to construct plot area
import seaborn as sns  #for sstatistical data visulaization 
from PIL import Image, ImageDraw, ImageFilter
#%matplotlib inline #to plot data inline in the notebook itself
df_all = pd.concat((train, test), axis=0, ignore_index=True)


In [ ]:
print(df_all)

In [ ]:
list(df_all.columns.values)

In [ ]:
#fill_value for nan`s 
stores_hm = df_all.pivot_table(index='shop_id', columns='item_category_id', values='item_cnt_month', aggfunc='count', fill_value=0)
#fig, ax = plt.subplots(figsize=(10,10))
#_ = sns.heatmap(stores_hm, ax=ax, cbar=False)

In [ ]:
stores_hm.tail()

In [ ]:
#This plot is for count of  products(items) sold under category for different shops in all years including predictabl yr 2015 nov
%matplotlib inline
fig, ax = plt.subplots(figsize=(10,10))
#generating heat map
ax = sns.heatmap(stores_hm, ax=ax, cbar=True) #cbar means colorbar , az means matplotlib Axes (optional)

In [ ]:
#this plot contains count of items sold under particular category for certain shops on nov 2015 (predictable month)
stores_hm = test.pivot_table(index='shop_id', columns='item_category_id', values='item_cnt_month', aggfunc='count', fill_value=0) #here in values we can use any item_cnt_month or item_cnt_prev_month
%matplotlib inline
fig, ax = plt.subplots(figsize=(10,10))
#generating heat map
ax = sns.heatmap(stores_hm, ax=ax, cbar=True) #cbar means colorbar , az means matplotlib Axes (optional)

In [ ]:
stores_hm.tail()

In [ ]:
train_1 = train
test_1 = test

In [ ]:
wordcld = pd.Series(test_1['item_name'].tolist()).astype(str)

In [ ]:
print(wordcld)

In [ ]:
from wordcloud import WordCloud
#wc = WordCloud(margin=1, max_font_size=100,background_color="yellow")
wc = WordCloud(margin=1, max_font_size=100,background_color=None,mode="RGBA")
wc.generate(' '.join(wordcld.astype(str)))

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(10,10))
plt.imshow(wc,interpolation='bilinear')

In [ ]:
print(test_1['shop_name'])

In [ ]:
for c in ['shop_name','item_name','item_category_name']:
    #Preprocessing
    #transforming the prediction target (Label Encoding)
    le = preprocessing.LabelEncoder()
    le.fit(list(train[c].unique())+list(test[c].unique()))
    #list(le.classes_)
    train[c] = le.fit_transform(train[c].astype(str))
    test[c] = le.fit_transform(test[c].astype(str))
    print(c)
    
    



In [ ]:
print(train['item_category_name'])

In [ ]:
#Training Models from train dataset
col = [c for c in train.columns if c not in ['item_cnt_month']]  #because item_cnt_month needs to be predicted on the basis of previous results
print(col)

In [ ]:
x1 = train[train['date_block_num']<33] #33 means before oct 2015
y1 = np.log1p(x1['item_cnt_month'].clip(0.,20.))  #result set 
#Clip (limit) the values in an array.Given an interval, values outside the interval are clipped to the interval edges. For example, if an interval of [0, 1] is specified, values smaller than 0 become 0, and values larger than 1 become 1.
x1 = x1[col] #without including result set 'item_cnt_month' (data before oct 2015)
print(x1)

In [ ]:
x2 = train[train['date_block_num']==33]  #33 means oct 2015
y2 = np.log1p(x2['item_cnt_month'].clip(0.,20.))
x2 = x2[col]
#print(x2)

In [ ]:
print(x2)

In [ ]:
#training on x1 and y1 i.e less then oct 2015`s data
reg = ensemble.ExtraTreesRegressor(n_estimators=25, n_jobs=-1, max_depth=15, random_state=18) #n_estimators(by default 10) no of trees in a forest, n_jobs=-1 means hen the number of jobs is set to the number of cores. #random_state is seed value
#Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks, that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression)
#ensemble learning on decsion trees
reg.fit(x1,y1) #building forest trees  #reg(model)
#Root Mean Square between train(true-train df(for oct 2015 i.e x2 y2)) and prediction
#error
#test for nov 2015`s  prediction on the basis of oct 2015 data 
#what we got accuracy is of about 0.8
print ('RMSE:',np.sqrt(metrics.mean_squared_error(y2.clip(0.,20.),reg.predict(x2).clip(0.,20.)))) #reg.predict - Predict regression target for X.


In [ ]:
print(train[col])

In [ ]:
#training on full training set(including oct 2015)
reg.fit(train[col],train['item_cnt_month'].clip(0.,20.))
test['item_cnt_month'] = 0.
test['item_cnt_month'] = reg.predict(test[col]).clip(0.,20.) # reg.predict- Predict regression target for X.#and put inside test file where item_cnt_month is zero
#test_1 = pd.merge(test_1,test,how='left',on=['item_id','shop_id','item_category_id'])
print(test[['ID','item_cnt_month']])
#print(test['item_cnt_month])
#test[['ID','shop_name','item_name','item_category_name','item_cnt_month']].to_csv('submission.csv', index=False)

In [ ]:
#test_1 = pd.merge(test_1,test,how='left',on=['item_id','shop_id','item_category_id'])
#test_2 = pd.merge(test[['shop_id','item_category_id','item_id','item_cnt_month']],test_1[['shop_id','item_category_id','item_id','shop_name','item_name','item_category_name']],how='left',on=['shop_id','item_category_id','item_id'])

In [ ]:
#print(test_1)

In [ ]:
test_1.columns

In [ ]:
print(test_1['shop_name_y'])

In [ ]:
#prdiction for nov 2015 
print(test[['ID','shop_name','item_name','item_category_name','item_cnt_month']])

In [ ]:
stores_hm =test.pivot_table(index='item_name', values='item_cnt_month', aggfunc='sum', fill_value=0)
%matplotlib inline
fig, ax = plt.subplots(figsize=(10,10))
#generating heat map
ax = sns.heatmap(stores_hm, ax=ax, cbar=True)
#ax = sns.barplot(x="item_name", y="Prediction(nov15)", data=stores_hm ,label="Total", color="b")

In [ ]:
wc.generate(' '.join(wordcld.astype(str)))